Mining!

In [65]:
from pyelasticsearch import ElasticSearch
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from pyquery import PyQuery as pq

In [12]:
client = ElasticSearch("http://localhost:9202")

In [13]:
client.count(index="new-ethereum-miner-transaction", doc_type="tx", query="blockNumber:[5000000 TO 6000000]")['count']

{'_shards': {'failed': 0, 'successful': 5, 'total': 5}, 'count': 1215719}

In [47]:
NUMBER_OF_JOBS = 1000

In [48]:
  def _count_by_object_or_string_query(client, query, index, doc_type):
    """
    Count objects in ElasticSearch by specified query

    Parameters
    ----------
    query : dict or str
        ElasticSearch query
    index : str
        ElasticSearch index
    doc_type : str
        Document type

    Returns
    -------
    int
        Number of objects in ElasticSearch
    """
    count_body = ''
    count_parameters = {}
    if type(query) is str:
      count_parameters['q'] = query
    else:
      count_body = {
        'query': query
      }
    return client.send_request('GET', [index, doc_type, '_count'], count_body, count_parameters)

  def iterate(client, index, doc_type, query, per=NUMBER_OF_JOBS):
    """
    Iterate through elasticsearch records

    Will return a chunk of records each time

    Parameters
    ----------
    index : str
        ElasticSearch index
    doc_type : str
        Document type
    query : dict or str
        ElasticSearch query
    per : int
        Max length of chunk

    Returns
    -------
    generator
        Generator that returns chunks with records by specified query
    """
    items_count = _count_by_object_or_string_query(client, query, index=index, doc_type=doc_type)['count']
    pages = round(items_count / per + 0.4999)
    scroll_id = None
    for page in tqdm(range(pages)):
      if not scroll_id:
        pagination_parameters = {'scroll': '60m', 'size': per}
        pagination_body = {}
        if type(query) is str:
          pagination_parameters['q'] = query
        else:
          pagination_body['query'] = query
        response = client.send_request('GET', [index, doc_type, '_search'], pagination_body, pagination_parameters)
        scroll_id = response['_scroll_id']
        page_items = response['hits']['hits']
      else:
        page_items = client.send_request('POST', ['_search', 'scroll'], {'scroll': '60m', 'scroll_id': scroll_id}, {})['hits']['hits']
      yield page_items


In [57]:
transactions = []
for t in iterate(client, query="blockNumber:[5000000 TO 6000000]", index="new-ethereum-miner-transaction", doc_type="tx"):
    transactions += t

In [59]:
transactions_df = pd.DataFrame([t["_source"] for t in transactions])

In [60]:
len(transactions_df["blockNumber"].unique())

1000001

In [90]:
top_miners_info = pd.DataFrame()

In [91]:
top_miners = transactions_df["author"].value_counts().head(10)

In [92]:
top_miners # TOP mining pools and miners here

0xea674fdde714fd979de3edf0f56aa9716b898ec8    312375
0x829bd824b016326a401d083b33d092293333a830    203344
0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5    184101
0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c    173429
0xb2930b35844a230f00e51431acae96fe543a0347    119791
0xf3b9d2c81f2b24b0fa0acaaa865b7d9ced5fc2fb     36338
0x2a65aca4d5fc5b5c859090a6c34d164135398226     31882
0x52e44f279f4203dcf680395379e5f9990a69f13c     16274
0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01     13980
0x84990f5d2e09f56cabdabf6409ad31bdd8363b50     11010
Name: author, dtype: int64

In [98]:
values = []
uncles = []
blocks = []
for miner in top_miners.index:
    values.append(transactions_df[transactions_df["author"] == miner]["value"].sum())
    uncles.append(transactions_df[(transactions_df["author"] == miner) & (transactions_df["rewardType"] == "uncle")].shape[0])
    blocks.append(transactions_df[(transactions_df["author"] == miner)  & (transactions_df["rewardType"] == "block")].shape[0])

In [99]:
top_miners_info["address"] = top_miners.index

In [101]:
top_miners_info["uncles"] = uncles

In [102]:
top_miners_info["blocks"] = blocks

In [104]:
top_miners_info["uncle rate"] = top_miners_info["uncles"] / (top_miners_info["blocks"] + top_miners_info["uncles"] )

In [105]:
top_miners_info

,address,values,uncles,blocks,uncle rate
0,0xea674fdde714fd979de3edf0f56aa9716b898ec8,912491.71875,41770,270605,0.133717
1,0x829bd824b016326a401d083b33d092293333a830,596135.06250,28583,174761,0.140565
2,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,514379.43750,53345,130756,0.289759
3,0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c,509111.90625,21879,151550,0.126155
4,0xb2930b35844a230f00e51431acae96fe543a0347,348182.81250,18715,101076,0.156230
5,0xf3b9d2c81f2b24b0fa0acaaa865b7d9ced5fc2fb,97935.28125,11851,24487,0.326132
6,0x2a65aca4d5fc5b5c859090a6c34d164135398226,91694.62500,6222,25660,0.195157
7,0x52e44f279f4203dcf680395379e5f9990a69f13c,48040.50000,1864,14410,0.114539
8,0x4bb96091ee9d802ed039c4d1a5f6216f90f81b01,40077.65625,2609,11371,0.186624
9,0x84990f5d2e09f56cabdabf6409ad31bdd8363b50,32243.81250,1518,9492,0.137875


There you can find top mining pools by mined ETH (values) and uncle rate (how many blocks are mined faster than anybody). Feel free to choose one of them for your profit!